In [1]:
from mainnet_launch.pages.risk_metrics.estimate_exit_liquidity_from_quotes import *
from mainnet_launch.constants import *

chain, base_asset, valid_autopools = (
    ETH_CHAIN,
    WETH,
    [a for a in ALL_AUTOPOOLS if a.chain == ETH_CHAIN and a.base_asset == WETH(ETH_CHAIN)],
)

unscaled_asset_exposure, percent_ownership_by_destination_df, token_df = fetch_needed_context(chain, valid_autopools)

tokemak_quote_requests_df, odos_quote_requests_df, raw_tokemak_quote_response_df, raw_odos_quote_response_df, df = (
    fetch_odos_and_tokemak_quotes(chain=chain, base_asset=base_asset, valid_autopools=valid_autopools)
)

Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 104/104 [01:08<00:00,  1.53it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 104/104 [00:04<00:00, 21.57it/s]


sleeping 2 minutes


Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 104/104 [01:08<00:00,  1.51it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 104/104 [00:06<00:00, 16.76it/s]


sleeping 2 minutes


Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 104/104 [01:07<00:00,  1.53it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 104/104 [00:06<00:00, 14.87it/s]

fetch_odos_and_tokemak_quotes took 465.6992 seconds.


In [ ]:


def _

In [ ]:
pio.templates.default = None


px.scatter(df, x="scaled_amount_in", y="effective_price", color="label")

In [17]:
reference_price = (
    df[df["scaled_amount_in"] == 5].groupby("token_in_symbol")["effective_price"].median().sort_values(ascending=False)
)
df["reference_price"] = df["token_in_symbol"].map(reference_price)
df["slippage_from_reference"] = (df["effective_price"] - df["reference_price"]) / df["reference_price"] * 100
df

,api,chain_id,token_in,token_out,token_in_symbol,token_out_symbol,scaled_amount_in,scaled_amount_out,datetime_received,pools_blacklist,aggregator_name,effective_price,reference_price,slippage_from_reference,label
0,odos,1,0xA35b1B31Ce002FBF2058D22F30f95D405200A15b,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,ETHx,WETH,20.0,21.267648,2025-08-07 19:22:09.983559+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.063382,1.064367,-0.092534,ETHx - odos
1,odos,1,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,pxETH,WETH,20.0,19.914820,2025-08-07 19:22:09.985212+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.995741,0.995944,-0.020429,pxETH - odos
2,odos,1,0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,weETH,WETH,600.0,642.953363,2025-08-07 19:22:09.998317+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.071589,1.071885,-0.027631,weETH - odos
3,odos,1,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,pxETH,WETH,1000.0,722.146083,2025-08-07 19:22:10.008571+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.722146,0.995944,-27.491329,pxETH - odos
4,odos,1,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,pxETH,WETH,500.0,492.201803,2025-08-07 19:22:10.027455+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.984404,0.995944,-1.158783,pxETH - odos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,tokemak,1,0xA35b1B31Ce002FBF2058D22F30f95D405200A15b,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,ETHx,WETH,20.0,21.293008,2025-08-07 19:28:32.160163+00:00,(),Odos,1.064650,1.064367,0.026597,ETHx - tokemak
620,tokemak,1,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,pxETH,WETH,600.0,596.925334,2025-08-07 19:28:33.247982+00:00,(),Odos,0.994876,0.995944,-0.107324,pxETH - tokemak
621,tokemak,1,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,wstETH,WETH,5.0,6.041838,2025-08-07 19:28:34.498037+00:00,(),Odos,1.208368,1.208423,-0.004580,wstETH - tokemak
622,tokemak,1,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,stETH,WETH,100.0,99.836204,2025-08-07 19:28:35.216400+00:00,(),Odos,0.998362,0.998434,-0.007167,stETH - tokemak


In [ ]:
reference_price = (
    df[df["scaled_amount_in"] == 10_000]
    .groupby("token_in_symbol")["effective_price"]
    .median()
    .sort_values(ascending=False)
)
df["reference_price"] = df["token_in_symbol"].map(reference_price)
df["slippage_from_reference"] = (df["effective_price"] - df["reference_price"]) / df["reference_price"] * 100
df


agg_df = (
    usdc.groupby(["label", "token_in_symbol", "scaled_amount_in"])
    .agg(effective_price=("effective_price", "median"))
    .reset_index()
)


agg_df["reference_price"] = agg_df["token_in_symbol"].map(reference_price)
agg_df["slippage_bps_from_reference"] = (
    (agg_df["reference_price"] - agg_df["effective_price"]) / agg_df["reference_price"] * 10_000
)

agg_df.reset_index()

,index,label,token_in_symbol,scaled_amount_in,effective_price,reference_price,slippage_bps_from_reference
0,0,ETHx - odos,ETHx,5.0,1.064023,1.064367,3.237680
1,1,ETHx - odos,ETHx,20.0,1.063374,1.064367,9.336106
2,2,ETHx - odos,ETHx,50.0,1.062187,1.064367,20.481296
3,3,ETHx - odos,ETHx,100.0,1.061415,1.064367,27.739895
4,4,ETHx - odos,ETHx,200.0,1.060415,1.064367,37.128701
...,...,...,...,...,...,...,...
203,203,wstETH - tokemak,wstETH,600.0,1.208333,1.208423,0.741908
204,204,wstETH - tokemak,wstETH,700.0,1.208324,1.208423,0.819302
205,205,wstETH - tokemak,wstETH,800.0,1.208315,1.208423,0.896939
206,206,wstETH - tokemak,wstETH,900.0,1.208308,1.208423,0.955575


In [25]:
px.scatter(agg_df, x="scaled_amount_in", y="slippage_bps_from_reference", color="label")

# the difference between the odos quote and the toekmak quote

(assuming odos is good at this pricing)

is because 

In [26]:
from mainnet_launch.pages.risk_metrics.estimate_exit_liquidity_from_quotes import *
from mainnet_launch.constants import *

chain, base_asset, valid_autopools = (
    BASE_CHAIN,
    USDC,
    [a for a in ALL_AUTOPOOLS if a.chain == BASE_CHAIN and a.base_asset == USDC(BASE_CHAIN)],
)

unscaled_asset_exposure, percent_ownership_by_destination_df, token_df = fetch_needed_context(chain, valid_autopools)

(
    tokemak_quote_requests_df,
    odos_quote_requests_df,
    raw_tokemak_quote_response_df,
    raw_odos_quote_response_df,
    usdc_df,
) = fetch_odos_and_tokemak_quotes(chain=chain, base_asset=base_asset, valid_autopools=valid_autopools)
usdc_df2 = usdc_df.copy()
usdc_df

Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 10/10 [00:04<00:00,  2.11it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 10/10 [00:01<00:00,  6.16it/s]


sleeping 2 minutes


Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 10/10 [00:04<00:00,  2.37it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 10/10 [00:03<00:00,  3.10it/s]


sleeping 2 minutes


Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 10/10 [00:03<00:00,  2.55it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 10/10 [00:05<00:00,  1.79it/s]

fetch_odos_and_tokemak_quotes took 265.9049 seconds.


,api,chain_id,token_in,token_out,token_in_symbol,token_out_symbol,scaled_amount_in,scaled_amount_out,datetime_received,pools_blacklist,aggregator_name,effective_price
0,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,200000.0,199423.465314,2025-08-07 19:43:04.411131+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.997117
1,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,20000.0,21016.695897,2025-08-07 19:43:04.415230+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.050835
2,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,20000.0,19982.141704,2025-08-07 19:43:04.419696+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.999107
3,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,200000.0,209267.785776,2025-08-07 19:43:04.422869+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.046339
4,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,100000.0,104926.216793,2025-08-07 19:43:04.524412+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.049262
5,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,50000.0,49945.885801,2025-08-07 19:43:04.564463+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.998918
6,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,10000.0,10510.005131,2025-08-07 19:43:04.568207+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.051001
7,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,50000.0,52515.035467,2025-08-07 19:43:04.679508+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.050301
8,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,10000.0,9992.430118,2025-08-07 19:43:05.112601+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.999243
9,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,100000.0,99866.206362,2025-08-07 19:43:05.210060+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.998662


In [29]:
usdc_df["label"] = usdc_df["token_in_symbol"] + " - " + df["api"]

agg_df = (
    usdc_df.groupby(["label", "token_in_symbol", "scaled_amount_in"])
    .agg(effective_price=("effective_price", "median"))
    .reset_index()
)


agg_df["reference_price"] = agg_df["token_in_symbol"].map(reference_price)
agg_df["slippage_bps_from_reference"] = (
    (agg_df["reference_price"] - agg_df["effective_price"]) / agg_df["reference_price"] * 10_000
)

agg_df.reset_index()

,index,label,token_in_symbol,scaled_amount_in,effective_price,reference_price,slippage_bps_from_reference
0,0,GHO - odos,GHO,10000.0,0.998323,NaN,NaN
1,1,GHO - odos,GHO,20000.0,0.998248,NaN,NaN
2,2,GHO - odos,GHO,50000.0,0.998153,NaN,NaN
3,3,GHO - odos,GHO,100000.0,0.998025,NaN,NaN
4,4,GHO - odos,GHO,200000.0,0.997246,NaN,NaN
5,5,scrvUSD - odos,scrvUSD,10000.0,1.049850,NaN,NaN
6,6,scrvUSD - odos,scrvUSD,20000.0,1.049684,NaN,NaN
7,7,scrvUSD - odos,scrvUSD,50000.0,1.049151,NaN,NaN
8,8,scrvUSD - odos,scrvUSD,100000.0,1.048116,NaN,NaN
9,9,scrvUSD - odos,scrvUSD,200000.0,1.045202,NaN,NaN


In [30]:
px.scatter(agg_df, x="scaled_amount_in", y="slippage_bps_from_reference", color="label")